# TensorFlow essentials

### 1. Import TensorFlow and Numpy:

In [1]:
import tensorflow as tf
import numpy as np

### 2. Defining tensors

A tensor is a generalization of a matrix that specifies an element by an arbitrary number of indices.
The syntax for tensors is even more nested vectors. For example, a 2-by-3-by-2 tensor is [[[1,2], [3,4], [5,6]], [[7,8], [9,10], [11,12]]], which can be thought of as two matrices, each of size 3-by-2. Consequently, we say this tensor has a rank of 3.

In [7]:
m1 = [[1.0, 2.0], 
      [3.0, 4.0]]

m2 = np.array([[1.0, 2.0], 
               [3.0, 4.0]], dtype=np.float32)

m3 = tf.constant([[1.0, 2.0], 
                  [3.0, 4.0]])

# Let's see what happens when we print them:
print(type(m1))
print(type(m2))
print(type(m3))

# By the way, there's a function called convert_to_tensor(...) that does exactly what you might expect.
# Let's use it to create tensor objects out of various types:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)

print(type(t1))
print(type(t2))
print(type(t3))

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


### 3. Evaluating ops 

In [11]:
x = tf.constant([[1, 2]])             # Start with a 1x2 matrix

neg_x = tf.negative(x)                # Let's negate it
print(neg_x)                          # It doesn't even perform the negation computation if you print.

with tf.Session() as sess:            # You need to summon a session so you can launch the negation op:
    result = sess.run(neg_x)
    print(result)

Tensor("Neg_5:0", shape=(1, 2), dtype=int32)
[[-1 -2]]


### 4. Interactive session

In [12]:
sess = tf.InteractiveSession()       # Interactive sessions are another way to use a session.

# We have a matrix we want to invert:
x = tf.constant([[1., 2.]])
neg_op = tf.negative(x)

# Since we're using an interactive session, we can just call the eval() method on the op.

result = neg_op.eval()
print(result)

sess.close()

[[-1. -2.]]


### 5. Session logging

In [10]:
x = tf.constant([[1, 2]])
neg_op = tf.negative(x)

# Start the session with a special config passed in to the constructor to enable logging
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_op)
    print(result)

[[-1 -2]]


### 6. Using variables

In [13]:
sess = tf.InteractiveSession()

# Below is a series of numbers. Don't worry what they mean. Just for fun, let's think of them as neural activations.
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]

spike = tf.Variable(False)          # Create a boolean variable called spike to detect a sudden increase in the values.
spike.initializer.run()             # All variables must be initialized, so do that by calling run() on its initializer

# Loop through the data and update the spike variable when there is a significant increase:
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, tf.constant(True))
        updater.eval()
    else:
        tf.assign(spike, False).eval()
    print("Spike", spike.eval())

sess.close()

Spike False
Spike True
Spike False
Spike False
Spike True
Spike False
Spike True


### 7. Saving and Loading variables

In [22]:
sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name='spikes')
spikes.initializer.run()

saver = tf.train.Saver()                # The saver op will enable saving and restoring

# do some operation
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        spikes_val = spikes.eval()
        spikes_val[i] = True
        updater = tf.assign(spikes, spikes_val)
        updater.eval()

# save session to disk
#save_path = saver.save(sess,"spikes.ckpt")
#print("spikes data saved in file: %s" % save_path)

sess.close()


'''
Lets Load the data again
'''
sess = tf.InteractiveSession()
spikes = tf.Variable([False]*8, name='spikes')
saver = tf.train.Saver()

try:
    saver.restore(sess, 'spikes.ckpt')
    print(spikes.eval())
except:
    print('file not found')

sess.close()

INFO:tensorflow:Restoring parameters from spikes.ckpt
file not found
